In [4]:
from typing import List, Dict, Optional
from pathlib import Path
import requests
import pandas as pd

class BioCycAPI:
    def __init__(self, email: str, password: str, database: str = "ECOLI"):
        """
        Initialize BioCyc API client
        
        Args:
            email (str): Your BioCyc account email
            password (str): Your BioCyc account password 
            database (str): BioCyc database to use (default: "ECOLI")
        """
        self.database = database
        self.base_url = "https://websvc.biocyc.org"
        
        # Create session and login
        self.session = requests.Session()
        login_response = self.session.post(
            f"{self.base_url}/credentials/login/",
            data={"email": email, "password": password}
        )
        if login_response.status_code != 200:
            raise Exception(f"Failed to login to BioCyc. Status code: {login_response.status_code}\nResponse: {login_response.text}")
            
    def get_smarttable(self, table_id: str, fmt: str = "tsv") -> pd.DataFrame:
        """
        Retrieve a smarttable from BioCyc
        
        Args:
            table_id (str): ID of the smarttable to retrieve
            fmt (str): Format to retrieve ('tsv', 'json', or 'xml')
            
        Returns:
            pd.DataFrame: Retrieved smarttable data
        """
        response = self.session.get(
            f"{self.base_url}/st-get",
            params={"id": table_id, "format": fmt}
        )
        
        if response.status_code != 200:
            raise Exception(f"Failed to retrieve smarttable: {response.text}")
            
        if fmt == "tsv":
            return pd.read_csv(response.text, sep="\t")
        elif fmt == "json":
            return pd.DataFrame(response.json())
        else:
            raise ValueError("Only tsv and json formats are supported for DataFrame conversion")

    def create_smarttable(self, name: str, query: str) -> str:
        """
        Create a new SmartTable using a query
        
        Args:
            name (str): Name for the new SmartTable
            query (str): BioCyc query to generate the SmartTable
                Example: "[x:Proteins] of 'ECOLI'" for all E. coli proteins
                
        Returns:
            str: ID of the created SmartTable
        """
        response = self.session.post(
            f"{self.base_url}/smarttables/create",
            params={
                "name": name,
                "query": query,
                "organism": self.database
            }
        )
        
        if response.status_code != 200:
            raise Exception(
                f"Failed to create smarttable:\n"
                f"Status code: {response.status_code}\n"
                f"URL: {response.url}\n"
                f"Response: {response.text}"
            )
        
        return response.text  # Should return the SmartTable ID

In [5]:
# Create a SmartTable of all E. coli proteins
api = BioCycAPI(email="cdalldor@ucsd.edu", password="BiY37pq8vB")

table_id = api.create_smarttable(
    name="ECOLI_Proteins",
    query="[x:Proteins] of 'ECOLI'"
)

# Then retrieve it
df = api.get_smarttable(table_id)

Exception: Failed to create smarttable:
Status code: 404
URL: https://websvc.biocyc.org/smarttables/create?name=ECOLI_Proteins&query=%5Bx%3AProteins%5D+of+%27ECOLI%27&organism=ECOLI
Response: <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html LANG="en"><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8">

<script src="https://code.jquery.com/jquery-3.5.1.min.js"></script>
<script src="https://code.jquery.com/jquery-migrate-3.3.2.min.js"></script>
<script>
var $j = jQuery.noConflict();
</script>

<!-- 
<link rel="stylesheet" type="text/css" href="/yui/build/assets/skins/sam/skin.css"> 
<link rel="stylesheet" type="text/css" href="/style.css" >

User web site style sheet customization should use the following file:
<link rel="stylesheet" type="text/css" href="/userWebsiteCustomization.css">

The previous 4 .css files are combined at web start up time by the Lisp fn publish-combined-static-files generating the 

In [8]:
api = BioCycAPI(email="cdalldor@ucsd.edu", password="BiY37pq8vB")
df = api.get_smarttable("biocyc14-2762-3579243971")

Exception: Failed to retrieve smarttable: {"error":"HTTP error #S(RESPONSE :NUMBER 404 :DESC Not Found):\nNo SmartTable biocyc14-2762-3579243971\n\n"}